In [105]:
sc

In [5]:
import csv

In [160]:
spark

In [161]:
sqlContext

In [181]:
data =  sc.textFile('coalesce_joined_weather_collisions_sorted_fixed.csv')
df = spark.read.load('coalesce_joined_weather_collisions_sorted_fixed.csv', format='csv', header=True, inferSchema=False)
df.cache()


df.registerTempTable('df')



from pyspark.sql.types import IntegerType

inj_kil = ["NUMBER_OF_CYCLIST_INJURED",
"NUMBER_OF_CYCLIST_KILLED",
"NUMBER_OF_MOTORIST_INJURED",
"NUMBER_OF_MOTORIST_KILLED",
"NUMBER_OF_PEDESTRIANS_INJURED",
"NUMBER_OF_PEDESTRIANS_KILLED",
"NUMBER_OF_PERSONS_INJURED",
"NUMBER_OF_PERSONS_KILLED"]

for x in inj_kil:
    df = df.withColumn(x, df[x].cast(IntegerType()))




In [ ]:
if __name__=='__main__':
    sc = SparkContext()
    spark = SparkSession(sc)
    sqlContext = SQLContext(sc)    
    df = spark.read.load('hdfs:///user/dzeng01/project/joined_weather_collisions_sorted_fixed.csv', format='csv', header=True, inferSchema=False)
temp = spark.sql(query)    

In [60]:
df.take(2)

[Row(BOROUGH=u'QUEENS', CONTRIBUTING_FACTOR_VEHICLE_1=u'Unspecified', CONTRIBUTING_FACTOR_VEHICLE_2=u'Unspecified', CONTRIBUTING_FACTOR_VEHICLE_3=None, CONTRIBUTING_FACTOR_VEHICLE_4=None, CONTRIBUTING_FACTOR_VEHICLE_5=None, CROSS_STREET_NAME=u'70 DRIVE', DATETIME=u'2016-06-20 21:06:00', LATITUDE=None, LOCATION=None, LONGITUDE=None, NUMBER_OF_CYCLIST_INJURED=u'0', NUMBER_OF_CYCLIST_KILLED=u'0', NUMBER_OF_MOTORIST_INJURED=u'0', NUMBER_OF_MOTORIST_KILLED=u'0', NUMBER_OF_PEDESTRIANS_INJURED=u'0', NUMBER_OF_PEDESTRIANS_KILLED=u'0', NUMBER_OF_PERSONS_INJURED=u'0', NUMBER_OF_PERSONS_KILLED=u'0', OFF_STREET_NAME=None, ON_STREET_NAME=u'METROPOLITAN AVENUE', UNIQUE_KEY=u'3465402', VEHICLE_TYPE_CODE_1=u'PASSENGER VEHICLE', VEHICLE_TYPE_CODE_2=u'PASSENGER VEHICLE', VEHICLE_TYPE_CODE_3=None, VEHICLE_TYPE_CODE_4=None, VEHICLE_TYPE_CODE_5=None, ZIP_CODE=u'11375', _c28=u'58671', Conditions=u'Clear', DateUTC=u'2016-06-21 01:51:00', DatetimeEDT=u'2016-06-20 21:51:00', Dew_PointF=u'63.0', Events=None, Fr

In [182]:
df.dtypes

[('BOROUGH', 'string'),
 ('CONTRIBUTING_FACTOR_VEHICLE_1', 'string'),
 ('CONTRIBUTING_FACTOR_VEHICLE_2', 'string'),
 ('CONTRIBUTING_FACTOR_VEHICLE_3', 'string'),
 ('CONTRIBUTING_FACTOR_VEHICLE_4', 'string'),
 ('CONTRIBUTING_FACTOR_VEHICLE_5', 'string'),
 ('CROSS_STREET_NAME', 'string'),
 ('DATETIME', 'string'),
 ('LATITUDE', 'string'),
 ('LOCATION', 'string'),
 ('LONGITUDE', 'string'),
 ('NUMBER_OF_CYCLIST_INJURED', 'int'),
 ('NUMBER_OF_CYCLIST_KILLED', 'int'),
 ('NUMBER_OF_MOTORIST_INJURED', 'int'),
 ('NUMBER_OF_MOTORIST_KILLED', 'int'),
 ('NUMBER_OF_PEDESTRIANS_INJURED', 'int'),
 ('NUMBER_OF_PEDESTRIANS_KILLED', 'int'),
 ('NUMBER_OF_PERSONS_INJURED', 'int'),
 ('NUMBER_OF_PERSONS_KILLED', 'int'),
 ('OFF_STREET_NAME', 'string'),
 ('ON_STREET_NAME', 'string'),
 ('UNIQUE_KEY', 'string'),
 ('VEHICLE_TYPE_CODE_1', 'string'),
 ('VEHICLE_TYPE_CODE_2', 'string'),
 ('VEHICLE_TYPE_CODE_3', 'string'),
 ('VEHICLE_TYPE_CODE_4', 'string'),
 ('VEHICLE_TYPE_CODE_5', 'string'),
 ('ZIP_CODE', 'string')

In [41]:
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
def filter_general(x):
    if x > 65:
        return float(x)
    else:
        return float(0)
    
gfg = udf(filter_general, FloatType())
newcol = df.withColumn("Dew_PointF", gfg("Dew_PointF"))
newcol.take(5)


[Row(BOROUGH=u'QUEENS', CONTRIBUTING_FACTOR_VEHICLE_1=u'Unspecified', CONTRIBUTING_FACTOR_VEHICLE_2=u'Unspecified', CONTRIBUTING_FACTOR_VEHICLE_3=None, CONTRIBUTING_FACTOR_VEHICLE_4=None, CONTRIBUTING_FACTOR_VEHICLE_5=None, CROSS_STREET_NAME=u'70 DRIVE', DATETIME=u'2016-06-20 21:06:00', LATITUDE=None, LOCATION=None, LONGITUDE=None, NUMBER_OF_CYCLIST_INJURED=u'0', NUMBER_OF_CYCLIST_KILLED=u'0', NUMBER_OF_MOTORIST_INJURED=u'0', NUMBER_OF_MOTORIST_KILLED=u'0', NUMBER_OF_PEDESTRIANS_INJURED=u'0', NUMBER_OF_PEDESTRIANS_KILLED=u'0', NUMBER_OF_PERSONS_INJURED=u'0', NUMBER_OF_PERSONS_KILLED=u'0', OFF_STREET_NAME=None, ON_STREET_NAME=u'METROPOLITAN AVENUE', UNIQUE_KEY=u'3465402', VEHICLE_TYPE_CODE_1=u'PASSENGER VEHICLE', VEHICLE_TYPE_CODE_2=u'PASSENGER VEHICLE', VEHICLE_TYPE_CODE_3=None, VEHICLE_TYPE_CODE_4=None, VEHICLE_TYPE_CODE_5=None, ZIP_CODE=u'11375', _c28=u'58671', Conditions=u'Clear', DateUTC=u'2016-06-21 01:51:00', DatetimeEDT=u'2016-06-20 21:51:00', Dew_PointF=63.0, Events=None, Freez

DataFrame[BOROUGH: string, CONTRIBUTING_FACTOR_VEHICLE_1: string, CONTRIBUTING_FACTOR_VEHICLE_2: string, CONTRIBUTING_FACTOR_VEHICLE_3: string, CONTRIBUTING_FACTOR_VEHICLE_4: string, CONTRIBUTING_FACTOR_VEHICLE_5: string, CROSS_STREET_NAME: string, DATETIME: string, LATITUDE: string, LOCATION: string, LONGITUDE: string, NUMBER_OF_CYCLIST_INJURED: string, NUMBER_OF_CYCLIST_KILLED: string, NUMBER_OF_MOTORIST_INJURED: string, NUMBER_OF_MOTORIST_KILLED: string, NUMBER_OF_PEDESTRIANS_INJURED: string, NUMBER_OF_PEDESTRIANS_KILLED: string, NUMBER_OF_PERSONS_INJURED: string, NUMBER_OF_PERSONS_KILLED: string, OFF_STREET_NAME: string, ON_STREET_NAME: string, UNIQUE_KEY: string, VEHICLE_TYPE_CODE_1: string, VEHICLE_TYPE_CODE_2: string, VEHICLE_TYPE_CODE_3: string, VEHICLE_TYPE_CODE_4: string, VEHICLE_TYPE_CODE_5: string, ZIP_CODE: string, _c28: string, Conditions: string, DateUTC: string, DatetimeEDT: string, Dew_PointF: string, Events: string, Freezing: string, Gust_SpeedMPH: string, Humidity: s

In [183]:
# sqlContext.sql('select * from df where Rain=1')

from calendar import monthrange
import json


#date = ""

# Yearly statistics


def months(x):
    month = {1:"January",
             2:"Feburary",
             3:"March",
             4:"April",
             5:"May",
             6:"June",
             7:"July",
             8:"August",
             9:"September",
             10:"October",
             11:"November",
             12:"December"}
    return str(month[x])




for yr in range(16,18):
    
    #### This should be code that will read in each csv for each year and register as df.
    stats = {}
    stats["0_YEAR"] = int(str(20)+str(yr))   
    
    
    ## Overall Statistics
    stats['Overall'] = {}
    m_arr = []
    mi_arr = []
    
    """PREPROCESSING"""
    for x in range(1,10):
        date = "20{1}-0{0}%".format(x, yr)
        query = 'select * from df where DATETIME like "{0}"'.format(date)
        m_arr.append((x, sqlContext.sql(query).count()))

    for x in range(10,13):
        date = "20{1}-{0}%".format(x, yr)
        query = 'select * from df where DATETIME like "{0}"'.format(date)
        m_arr.append((x, sqlContext.sql(query).count()))
        
    for x in range(1,10):
        date = "20{1}-0{0}%".format(x, yr)
        query = 'select * from df where DATETIME like "{0}" and ( \
                                                                NUMBER_OF_CYCLIST_INJURED!=0 or \
                                                                NUMBER_OF_MOTORIST_INJURED!=0 or \
                                                                NUMBER_OF_PEDESTRIANS_INJURED!=0 or \
                                                                NUMBER_OF_PERSONS_INJURED!=0)'.format(date)
        mi_arr.append((x, sqlContext.sql(query).count()))

    for x in range(10,13):
        date = "20{1}-{0}%".format(x, yr)
        query = 'select * from df where DATETIME like "{0}" and ( \
                                                                NUMBER_OF_CYCLIST_INJURED!=0 or \
                                                                NUMBER_OF_MOTORIST_INJURED!=0 or \
                                                                NUMBER_OF_PEDESTRIANS_INJURED!=0 or \
                                                                NUMBER_OF_PERSONS_INJURED!=0)'.format(date)
        mi_arr.append((x, sqlContext.sql(query).count()))
    
    injury_total = sum([x[1] for x in mi_arr])
    
    total = sum([x[1] for x in m_arr])
    
    
    # Statistics
    stats['Overall']['Total Number of Collisions in Year'] = total
    stats['Overall']['Total Number of Collisions w/ Injury in Year'] = injury_total
    stats['Overall']['Average Collisions Per Month'] = total // 12
    stats['Overall']['Average Collisions Per Day'] = total // 365
    stats['Overall']['Monthly Adjusted Average Weight (%)'] = [str((months(x[0]), round((float(x[1])/float(total)*100.0),2))) for x in m_arr]
    stats['Overall']['Collisions Per Month'] = [str((months(x[0]), x[1])) for x in m_arr]
    stats['Overall']['Average Collisions Per Day (Monthly)'] = [str((months(x), m_arr[x-1][1]/monthrange(int(str(20)+str(yr)), x)[1])) for x in range(1,13)]
    stats['Overall']['Injury Rate (Year %)'] = float(injury_total)/float(total) * 100.00
    stats['Overall']['Number of Collisions w/ Injuries Per Month'] = [str((months(x[0]), x[1])) for x in mi_arr]
    stats['Overall']['Injury Rate Per Month (Month %)'] = [str((months(x+1), round((float(mi_arr[x][1])/float(m_arr[x][1]) * 100.00),2))) for x in range(0,12) if m_arr[x][1] != 0]
    
    # Rain Statistics
    stats['Rain'] = {}
    r_arr = []
    ri_arr = []

    """PREPROCESSING"""
    for x in range(1,10):
        date = "20{1}-0{0}%".format(x, yr)
        query = 'select * from df where Rain="1" and DATETIME like "{0}"'.format(date)
        r_arr.append((x, sqlContext.sql(query).count()))

    for x in range(10,13):
        date = "20{1}-{0}%".format(x, yr)
        query = 'select * from df where Rain="1" and DATETIME like "{0}"'.format(date)
        r_arr.append((x, sqlContext.sql(query).count()))
    rain_total = sum([x[1] for x in r_arr])
    
    
    for x in range(1,10):
        date = "20{1}-0{0}%".format(x, yr)
        query = 'select * from df where Rain="1" and DATETIME like "{0}" and ( \
                                                                NUMBER_OF_CYCLIST_INJURED!=0 or \
                                                                NUMBER_OF_MOTORIST_INJURED!=0 or \
                                                                NUMBER_OF_PEDESTRIANS_INJURED!=0 or \
                                                                NUMBER_OF_PERSONS_INJURED!=0)'.format(date)
        ri_arr.append((x, sqlContext.sql(query).count()))

    for x in range(10,13):
        date = "20{1}-{0}%".format(x, yr)
        query = 'select * from df where Rain="1" and DATETIME like "{0}" and ( \
                                                                NUMBER_OF_CYCLIST_INJURED!=0 or \
                                                                NUMBER_OF_MOTORIST_INJURED!=0 or \
                                                                NUMBER_OF_PEDESTRIANS_INJURED!=0 or \
                                                                NUMBER_OF_PERSONS_INJURED!=0)'.format(date)
        ri_arr.append((x, sqlContext.sql(query).count()))
    rain_injury_total = sum([x[1] for x in ri_arr])
    
    # Statistics
    stats['Rain']['Total Number of Collisions w/ Injury & Rain in Year'] = rain_injury_total
    stats['Rain']['Injury Rate w/ Rain (Year %)'] = float(rain_injury_total)/float(rain_total) * 100.00
    stats['Rain']['Total Number of Rain Collisions in Year'] = rain_total
    stats['Rain']['Percent of Collisions in Rain Over Year (%)'] = (float(rain_total) / float(total)) * 100.00
    stats['Rain']['Collisions in Rain'] = [str((months(x[0]), x[1])) for x in r_arr]
    stats['Rain']['Average Collisions in Rain Per Month'] = rain_total//12
    stats['Rain']['Number of Collisions w/ Injuries and Rain Per Month'] = [str((months(x[0]), x[1])) for x in ri_arr]
    stats['Rain']['Percent of Collisions in Each Month That Are Rain Collisions (Rain %)'] = [str((months(x+1), round((float(r_arr[x][1])/float(m_arr[x][1]))*100.0, 2))) for x in range(0,12) if m_arr[x][1] != 0]
    stats['Rain']['Injury Rate Per Month w/ Rain (Month %)'] = [str((months(x+1), round((float(ri_arr[x][1])/float(r_arr[x][1]) * 100.00),2))) for x in range(0,12) if r_arr[x][1] != 0]
    
    
    
    print json.dumps(stats, indent=4, sort_keys=True)
    #print stats


# print arr

# Monthly Statistics

# for x in [date]:
    
#     a = sqlContext.sql('select * from df where Rain=1')
#     a.coalesce(1).write.csv('rain', header=True)

#     a = sqlContext.sql('select * from df where Snow=1')
#     a.coalesce(1).write.csv('snow', header=True)

#     a = sqlContext.sql('select * from df where Freezing=1')
#     a.coalesce(1).write.csv('freezing', header=True)

#     a = sqlContext.sql('select * from df where LowVisibility=1')
#     a.coalesce(1).write.csv('foggy', header=True)

#     a = sqlContext.sql('select * from df where Windy=1')
#     a.coalesce(1).write.csv('windy', header=True)


# df = df.filter(df.LATITUDE != '0')

# from pyspark.sql.functions import col


# a = df.where(col("LOCATION").isNotNull())
# a.coalesce(1).write.csv('geodata', header=True)

{
    "0_YEAR": 2016, 
    "Overall": {
        "Average Collisions Per Day": 295, 
        "Average Collisions Per Day (Monthly)": [
            "('January', 0)", 
            "('Feburary', 0)", 
            "('March', 0)", 
            "('April', 0)", 
            "('May', 0)", 
            "('June', 0)", 
            "('July', 339)", 
            "('August', 634)", 
            "('September', 649)", 
            "('October', 634)", 
            "('November', 641)", 
            "('December', 616)"
        ], 
        "Average Collisions Per Month": 8974, 
        "Collisions Per Month": [
            "('January', 0)", 
            "('Feburary', 0)", 
            "('March', 0)", 
            "('April', 0)", 
            "('May', 0)", 
            "('June', 1)", 
            "('July', 10516)", 
            "('August', 19672)", 
            "('September', 19491)", 
            "('October', 19659)", 
            "('November', 19235)", 
            "('December', 19118)"
        ], 
     

In [186]:
print(float(217417) / float(35408))

6.1403355174
